<a href="https://colab.research.google.com/github/hepuliu/Masters_Thesis/blob/sandbox_pink/sandbox_pink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Master Thesis Simulation Sandbox
Flood Prevention Dam Sizing with Machine Learining Approaches - Hepu Liu

### Overall Project Simulation Steps
1. Process discharge data from Waldangelbach Station

2. Process precipitation data from Baiertal  Station

3. Build Prediction Models

4. Evaluation of NSE

### Variable Naming Conventions

- Weather Stations Naming: ('p' for precipitation, 'd' for discharge, 'a' for different stations, 'r' for result)

  - da: Waldangelbach Station
  - pa: Baiertal Station
  - pr: combined/resulting precipitation
  - dr: predicted/resulting discharge

- Variable Naming Coventions: 
  - df: data frame
  - trs: training set
  - tes: testing set
  - lstm: LSTM
  - cnn: CNN
  - lstmss: LSTM-seq2sqe


## Importing Libraries

In [419]:
# importing libraries
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from fbprophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
from pandas import concat
from math import sqrt
from numpy import concatenate
from numpy import loadtxt
from numpy import array
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount

<function google.colab.drive.mount>

## Importing Datasets

In [420]:
# import datafram for CNN
df_cnn = pd.read_csv('/content/drive/MyDrive/thesis/dataset/cleaned_df/df_cnn.csv')
df_cnn

,ds,y,temp,rad,preci
0,2007-01-01 00:00:00,0.226,10.00,0.0,2.6
1,2007-01-01 01:00:00,0.248,10.58,0.0,0.8
2,2007-01-01 02:00:00,0.248,11.22,0.0,0.2
3,2007-01-01 03:00:00,0.320,11.42,0.0,0.6
4,2007-01-01 04:00:00,0.346,11.58,0.0,0.0
...,...,...,...,...,...
105148,2018-12-31 19:00:00,0.232,6.19,0.0,0.0
105149,2018-12-31 20:00:00,0.248,6.23,0.0,0.1
105150,2018-12-31 21:00:00,0.232,6.25,0.0,0.1
105151,2018-12-31 22:00:00,0.226,6.26,0.0,0.0


## Data Processing

In [421]:
# Data Processing for Multivariable CNN - Small Sample Size for Testing
df_cnn = df_cnn.iloc[:1500, :]
df_cnn = df_cnn.set_index('ds')
df_cnn = df_cnn[['temp', 'rad', 'preci', 'y']]
df_cnn = df_cnn.to_numpy()
df_cnn

array([[ 10.   ,   0.   ,   2.6  ,   0.226],
       [ 10.58 ,   0.   ,   0.8  ,   0.248],
       [ 11.22 ,   0.   ,   0.2  ,   0.248],
       ...,
       [  4.92 , 100.   ,   0.   ,   0.637],
       [  7.42 , 100.   ,   0.   ,   0.605],
       [ 10.12 , 100.   ,   0.   ,   0.574]])

In [422]:
# CNN - split into train and test sets
trs_cnn = df_cnn[:1000, :]
tes_cnn = df_cnn[1000:, :]

In [423]:
# CNN Data Processing - split multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps = 3
# convert training set into input/output
trs_cnn_X, trs_cnn_y = split_sequences(trs_cnn, n_steps)
print(trs_cnn_X.shape, trs_cnn_y.shape)
# convert testing set into input/output
tes_cnn_X, tes_cnn_y = split_sequences(tes_cnn, n_steps)
print(tes_cnn_X.shape, tes_cnn_y.shape)

# define number of feature variables
n_features = trs_cnn_X.shape[2]

(998, 3, 3) (998,)
(498, 3, 3) (498,)


## Prediction

In [424]:
# CNN Model Definition
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(optimizer='adam', loss='mse')

## Evaluation

In [425]:
# CNN Fit Model
cnn_history = model_cnn.fit(trs_cnn_X, trs_cnn_y, validation_data=(tes_cnn_X, tes_cnn_y), epochs=1000, verbose=0)


In [426]:
# CNN Prediction
trs_cnn_pred = model_cnn.predict(trs_cnn_X)
tes_cnn_pred = model_cnn.predict(tes_cnn_X)
# print('Train rmse:', np.sqrt(mean_squared_error(trs_cnn_y, trs_cnn_pred)))
# print('Validation rmse:', np.sqrt(mean_squared_error(tes_cnn_y, tes_cnn_pred)))


In [427]:
trs_cnn_pred = trs_cnn_pred.reshape(-1,1)
trs_cnn_y = trs_cnn_y.reshape(-1,1)
tes_cnn_pred = trs_cnn_pred.reshape(-1,1)
tes_cnn_y = trs_cnn_y.reshape(-1,1)

In [428]:
# calculate NSE
nse_trs_cnn = 1-(np.sum((trs_cnn_pred-trs_cnn_y)**2)/np.sum((trs_cnn_y-np.mean(trs_cnn_y))**2))
nse_tes_cnn = 1-(np.sum((tes_cnn_pred-tes_cnn_y)**2)/np.sum((tes_cnn_y-np.mean(tes_cnn_y))**2))

print('Train NSE: %.3f' % nse_trs_cnn)
print('Test NSE: %.3f' % nse_tes_cnn)

# # calculate RMSE
# rmse_trs_cnn = np.sqrt(mean_squared_error(trs_cnn_y, trs_cnn_pred))
# rmse_tes_cnn = np.sqrt(mean_squared_error(tes_cnn_y, tes_cnn_pred))
# print('Train RMSE: %.3f', % rmse_trs_cnn)
# print('Test RMSE: %.3f', % rmse_tes_cnn)

Train NSE: 0.482
Test NSE: 0.482


# Archive

In [429]:
# # Cleanup Discharge A DataFrame da_df
# da_df = pd.read_csv('/content/drive/MyDrive/thesis/dataset/Wiesloch_waldangelbach_hourly_20070101-20210501.csv')
# da_df = da_df.iloc[13:].reset_index(drop=True)
# da_df.columns = da_df.iloc[0]
# da_df = da_df.iloc[3:].reset_index(drop=True)
# da_df = da_df.iloc[:, 4:7] # precipitation unit [m3/s]
# da_df['Uhrzeit'] = da_df['Uhrzeit'].str.replace(' v', '')
# da_df['t'] = pd.to_datetime(da_df['Datum']+' '+da_df['Uhrzeit'], format=('%y-%m-%d %H:%M:%S'))
# da_df = da_df.iloc[:,2:]
# da_df.columns = ['discharge [m3/s]', 't']
# da_df = da_df[['t','discharge [m3/s]']]
# da_df.to_csv('/content/drive/MyDrive/thesis/dataset/cleaned_df/da_df.csv', index=False)

In [430]:
# # Cleanup Precipitation A DataFrame pa_df
# pa_df = pd.read_csv('/content/drive/MyDrive/thesis/dataset/Weather_station_Baiertal.csv')
# pa_df.columns = pa_df.iloc[0]
# pa_df = pa_df.iloc[1:].reset_index(drop=True)
# pa_df['t'] = pd.to_datetime(pa_df['date']+' '+pa_df['time'], format=('%y-%m-%d %H:%M'))
# pa_df = pa_df.iloc[:,2:]
# cols = list(pa_df.columns)
# cols = [cols[-1]] + cols[:-1]
# pa_df = pa_df[cols]
# pa_df.to_csv('/content/drive/MyDrive/thesis/dataset/cleaned_df/pa_df.csv', index=False)


In [431]:
# # Cleanup Precipitation B DataFrame pb_df
# pb_df = pd.read_csv('/content/drive/MyDrive/thesis/dataset/Weather_station_Stifterhof.csv')
# pb_df.columns = pb_df.iloc[0]
# pb_df = pb_df.iloc[1:].reset_index(drop=True)
# pb_df['t'] = pd.to_datetime(pb_df['date']+' '+pb_df['time'], format=('%y-%m-%d %H:%M'))
# pb_df = pb_df.iloc[:,2:]
# cols = list(pb_df.columns)
# cols = [cols[-1]] + cols[:-1]
# pb_df = pb_df[cols]
# pb_df.to_csv('/content/drive/MyDrive/thesis/dataset/cleaned_df/pb_df.csv', index=False)

In [432]:
# # Plot Line Graph 20000 row with GPU = 3mins
# def line_plot(df, title):
#   label_font = {'family':'serif', 'color':'black', 'size':'12'}
#   title_font = {'family':'serif', 'color':'black', 'size':'14'}
#   fig = plt.figure(figsize=(8,8))
#   plt.plot(df['ds'], df['yhat'])
#   plt.xlabel( 't', fontdict = label_font)
#   plt.ylabel( 'd', fontdict = label_font)
#   plt.title(title, fontdict = title_font)
   
# # line_plot(da_df, 'Discharge A')


In [433]:
# df_fbp = pd.read_csv('/content/drive/MyDrive/thesis/dataset/cleaned_df/df_fbp.csv')
# df_fbp.dtypes

In [434]:
# # import datasets
# da_dr_fbp = pd.read_csv('/content/drive/MyDrive/thesis/dataset/cleaned_df/da_dr_fbp.csv')
# da_dr_fbp.dtypes

In [435]:
# # Calculate NSE FBProphet
# da_dr_fbp['num']= (da_dr_fbp['y']-da_dr_fbp['yhat'])**2
# da_dr_fbp['denom']=(da_dr_fbp['y']-da_dr_fbp['y'].mean())**2
# NSE = 1 - (da_dr_fbp['num'].sum()/da_dr_fbp['denom'].sum())
# NSE

In [436]:
# # NSE Calculation and Plot

# o = np.array([1,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7])
# m = np.array([1.1,2.2,3.2,4,5,6.1,7.2,8.5,8,10.5,1,2,4,5,6,7])
# # nse = 1-(np.sum((p-t)**2)/np.sum((t-np.mean(t))**2))
# # print('Test NSE: %.3f' % nse)
# # plot


# fig= plt.figure(figsize=(14, 4))
# plt.title('Insert Title')
# plt.plot(o, label='observed', color='#00688b', linewidth=0.5)
# plt.plot(m, label='model', color='#ee7600', linewidth=0.5)
# plt.plot([], [], ' ', label='NSE = %.3f' % nse)
# plt.ylabel('y label')
# plt.ylabel('x label')
# plt.legend()
# plt.show()



In [437]:
# results_df = df_fbp.merge(da_dr_fbp, on='ds', how='left')
# results_df.to_csv('/content/drive/MyDrive/thesis/dataset/results/da_dr_fbp.csv', index=False)

In [438]:
# # import datasets
# da_dr_fbp = pd.read_csv('/content/drive/MyDrive/thesis/dataset/results/da_dr_fbp_2.csv')
# da_dr_fbp